<a href="https://colab.research.google.com/github/Biscuitkru/CS5228_Project/blob/main/CS5228_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q numpy
!pip install --upgrade -q scikit-learn
!pip install --upgrade -q xgboost
!pip install -q tqdm
!pip install -q joblib

In [3]:
import os, re, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import math
import json

from pathlib import Path
from datetime import datetime
from typing import Optional, Tuple, Dict, List, Any
from collections import Counter

# for modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

from tqdm.auto import tqdm
from lightgbm import LGBMRegressor

import joblib

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
########################################
# Configs
########################################

# Main
train_path = "/content/drive/MyDrive/CS Masters/CS5228/train.csv"
test_path = "/content/drive/MyDrive/CS Masters/CS5228/test.csv"

# Auxiliary data
hdb_data_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-hdb-block-details.csv"
gov_hawkers_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-gov-hawkers.csv"
mrt_stations_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-mrt-stations.csv"
pri_schools_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-primary-schools.csv"
sec_schools_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-secondary-schools.csv"
shopping_malls_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-shopping-malls.csv"

# Modeling controls
seed = 777
rng = np.random.default_rng(seed)

In [6]:
################################
# configs for local
################################

# local_path = r"C:\Users\admin\Desktop\school\MComp_AI\CS5228_Knowledge_Discovery_and_Data_Mining\Project\cs5228\CS5228_Project\data"


# # Main
# train_path = local_path + "/train.csv"
# test_path = local_path + "/test.csv"

# # Auxiliary data
# hdb_data_path = local_path + "/auxiliary-data/sg-hdb-block-details.csv"
# gov_hawkers_path = local_path + "/auxiliary-data/sg-gov-hawkers.csv"
# mrt_stations_path = local_path + "/auxiliary-data/sg-mrt-stations.csv"
# pri_schools_path = local_path + "/auxiliary-data/sg-primary-schools.csv"
# sec_schools_path = local_path + "/auxiliary-data/sg-secondary-schools.csv"
# shopping_malls_path = local_path + "/auxiliary-data/sg-shopping-malls.csv"

# # Modeling controls
# seed = 777
# np.random.seed(seed)

## Data Preprocessing

In [7]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [8]:
train_df.head()

,MONTH,TOWN,FLAT_TYPE,BLOCK,STREET,FLOOR_RANGE,FLOOR_AREA_SQM,FLAT_MODEL,ECO_CATEGORY,LEASE_COMMENCE_DATA,RESALE_PRICE
0,2020-10,woodlands,4 room,681B,woodlands drive 62,07 to 09,102.0,premium apartment,uncategorized,2000,420000.0
1,2021-07,bishan,4 room,264,bishan street 24,07 to 09,104.0,model a,uncategorized,1992,585000.0
2,2021-05,bukit panjang,4 room,520,jelapang road,19 to 21,102.0,model a,uncategorized,1998,450000.0
3,2021-08,punggol,4 room,121B,edgedale plains,16 to 18,93.0,model a,uncategorized,2017,465000.0
4,2023-05,hougang,5 room,997B,Buangkok Crescent,10 to 12,113.0,improved,uncategorized,2018,710000.0


In [9]:
train_df.columns = train_df.columns.str.lower()
test_df.columns = test_df.columns.str.lower()

In [10]:
train_df.describe() # No NA values at all

,floor_area_sqm,lease_commence_data,resale_price
count,162691.000000,162691.000000,1.626910e+05
mean,96.891250,1996.358993,5.188430e+05
std,24.030547,14.245920,1.832442e+05
min,31.000000,1966.000000,1.500000e+05
25%,82.000000,1985.000000,3.820000e+05
50%,93.000000,1996.000000,4.880000e+05
75%,112.000000,2011.000000,6.220000e+05
max,366.700000,2022.000000,1.658888e+06


In [11]:
# Date Manipulation
def process_month_column(df):
    df['month'] = pd.to_datetime(df['month'], format="%Y-%m")
    df['sale_year'] = df['month'].dt.year
    df['sale_month'] = df['month'].dt.month
    # sale_quarter as a feature to capture potential seasonality in resale prices.
    df['sale_quarter'] = df['month'].dt.quarter
    return df

train_df = process_month_column(train_df)
test_df = process_month_column(test_df)

# additional month_index to capture trend
abs_month_train = train_df["sale_year"]*12 + train_df["sale_month"]
abs_month_test  = test_df["sale_year"]*12 + test_df["sale_month"]
train_df["month_index"] = (abs_month_train - abs_month_train.min()).astype(float)
test_df["month_index"]  = (abs_month_test  - abs_month_train.min()).astype(float)

In [12]:
# To remove trailing whitespace and reduce multiple spaces to a single space
def norm_text_basic(s):
    if not isinstance(s, str):
        return s
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

# Normalise + uppercase, mainly for the str values
def norm_key(s, case="upper"):
    s = norm_text_basic(s)
    if isinstance(s, str):
        return s.upper() if case == "upper" else s.casefold()
    return s

In [13]:
for col in ["town","flat_type", "block", "street", "flat_model"]:
    if col in train_df.columns: train_df[col] = train_df[col].map(norm_key)
    if col in test_df.columns: test_df[col]  = test_df[col].map(norm_key)

In [14]:
train_df["flat_type"].unique()

array(['4 ROOM', '5 ROOM', '3 ROOM', '5-ROOM', '3-ROOM', 'EXECUTIVE',
       '4-ROOM', '2-ROOM', '2 ROOM', '1 ROOM', 'MULTI GENERATION',
       '1-ROOM'], dtype=object)

In [15]:
test_df["flat_type"].unique()

array(['3 ROOM', '4 ROOM', '5-ROOM', '5 ROOM', '4-ROOM', 'EXECUTIVE',
       '3-ROOM', '2 ROOM', '2-ROOM', '1-ROOM', 'MULTI GENERATION',
       '1 ROOM'], dtype=object)

In [16]:
# Standardize flat type
def standardise_flat_type(val: str) -> str:
    v = val.strip().replace("_", " ").replace("-", " ").replace("  ", " ")
    v = v.replace("ROOM", " ROOM")
    v = re.sub(r"\s+", " ", v).strip()
    return v

In [17]:
train_df['flat_type'] = train_df['flat_type'].apply(standardise_flat_type)
test_df['flat_type'] = test_df['flat_type'].apply(standardise_flat_type)
print(train_df['flat_type'].unique())

['4 ROOM' '5 ROOM' '3 ROOM' 'EXECUTIVE' '2 ROOM' '1 ROOM'
 'MULTI GENERATION']


In [18]:
# General Price hierarchy for HDB flats
flat_type_order = ["1 ROOM","2 ROOM","3 ROOM","4 ROOM","5 ROOM","MULTI GENERATION","EXECUTIVE"]
order_map = {ft:i for i, ft in enumerate(flat_type_order)}

train_df["flat_type_price_hier"] = train_df["flat_type"].map(lambda x: order_map.get(str(x).upper(), np.nan))
test_df["flat_type_price_hier"]  = test_df["flat_type"].map(lambda x: order_map.get(str(x).upper(), np.nan))

# For Decision trees
for col, val in [("is_exec","EXECUTIVE"), ("is_multigen","MULTI GENERATION")]:
    train_df[col] = (train_df["flat_type"].str.upper() == val).astype(int)
    test_df[col]  = (test_df["flat_type"].str.upper() == val).astype(int)

In [19]:
# No. of Bedrooms
BEDROOMS_MAP = {"1 ROOM":0,"2 ROOM":1,"3 ROOM":2,"4 ROOM":3,"5 ROOM":4,"EXECUTIVE":3,"MULTI GENERATION":4}
train_df["bedrooms_est"] = train_df["flat_type"].str.upper().map(BEDROOMS_MAP)
test_df["bedrooms_est"]  = test_df["flat_type"].str.upper().map(BEDROOMS_MAP)

In [20]:
# Floor Range, just going to set as the middle numnber
def parse_floor_mid(floor_range) -> float:
    if isinstance(floor_range, (float)):
        return float(floor_range)
    m = re.findall(r"\d{1,2}", floor_range)
    if len(m) >= 2:
        lo, hi = int(m[0]), int(m[1])
        if lo <= hi:
            return (lo + hi) / 2.0
    return np.nan

In [21]:
train_df['floor_range'] = train_df['floor_range'].apply(parse_floor_mid).astype(float)
train_df = train_df.rename(columns={"floor_range": "floor_mid"})

test_df['floor_range'] = test_df['floor_range'].apply(parse_floor_mid).astype(float)
test_df = test_df.rename(columns={"floor_range": "floor_mid"})

print(np.sort(train_df['floor_mid'].unique()))
print(np.sort(test_df['floor_mid'].unique()))

[ 2.  5.  8. 11. 14. 17. 20. 23. 26. 29. 32. 35. 38. 41. 44. 47. 50.]
[ 2.  5.  8. 11. 14. 17. 20. 23. 26. 29. 32. 35. 38. 41. 44. 47. 50.]


In [22]:
train_df['flat_model'].unique()

array(['PREMIUM APARTMENT', 'MODEL A', 'IMPROVED', 'NEW GENERATION',
       'STANDARD', 'MAISONETTE', 'DBSS', 'SIMPLIFIED', 'APARTMENT',
       'MODEL A2', 'TYPE S1', 'ADJOINED FLAT', 'MODEL A MAISONETTE',
       'TYPE S2', '3GEN', '2 ROOM', 'TERRACE', 'PREMIUM APARTMENT LOFT',
       'PREMIUM MAISONETTE', 'MULTI GENERATION', 'IMPROVED MAISONETTE'],
      dtype=object)

In [23]:
test_df['flat_model'].unique()

array(['MODEL A', 'IMPROVED', 'SIMPLIFIED', 'APARTMENT',
       'PREMIUM APARTMENT', 'NEW GENERATION', 'STANDARD', 'MAISONETTE',
       'DBSS', 'MODEL A MAISONETTE', 'MODEL A2', 'IMPROVED MAISONETTE',
       'TYPE S1', '2 ROOM', 'ADJOINED FLAT', '3GEN', 'TYPE S2',
       'PREMIUM APARTMENT LOFT', 'TERRACE', 'PREMIUM MAISONETTE',
       'MULTI GENERATION'], dtype=object)

In [24]:
print(train_df['eco_category'].unique())
print(test_df['eco_category'].unique())

# Current dataset is taken directly from the Kaggle scoreboard which is missing eco_category (Might need to double check whether this is true)
train_df = train_df.drop('eco_category', axis=1)
test_df = test_df.drop('eco_category', axis=1)

['uncategorized']
['uncategorized']


In [25]:
# Flat age, years left from 99
train_df = train_df.rename(columns={"lease_commence_data": "lease_commence_date"})
test_df = test_df.rename(columns={"lease_commence_data": "lease_commence_date"})

train_df["lease_left"] = 99 - (train_df["sale_year"] - train_df["lease_commence_date"])
test_df["lease_left"] = 99 - (test_df["sale_year"] - test_df["lease_commence_date"])

In [26]:
train_df.head()

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,sale_year,sale_month,sale_quarter,month_index,flat_type_price_hier,is_exec,is_multigen,bedrooms_est,lease_left
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,2020,10,4,45.0,3,0,0,3,79
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,2021,7,3,54.0,3,0,0,3,70
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,2021,5,2,52.0,3,0,0,3,76
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,2021,8,3,55.0,3,0,0,3,95
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,2023,5,2,76.0,4,0,0,4,94


In [27]:
# HDB price drops faster as the remaining lease falls—use non-linear transforms and an approximate leasehold-relativity curve (aka “Bala”).
# URA’s table pegs 99y ≈ 96%, 60y ≈ 80%, 30y ≈ 60% of freehold, https://www.ura.gov.sg/-/media/Corporate/Guidelines/Development-control/Circulars/2022/Jul/dc22-08-Appendix2.pdf
# A convex decay so a curved transform helps models learn this faster than a raw linear lease_left

for df in (train_df, test_df):
    df["lease_left_sq"]  = df["lease_left"]**2
    df["lease_left_log"] = np.log1p(df["lease_left"].clip(lower=0))

    # Bala-style approximation using anchor points (convex decay)
    anchors_years = np.array([30, 60, 99], dtype=float)
    anchors_rel   = np.array([0.60, 0.80, 0.96], dtype=float)
    df["lease_rel_approx"] = np.interp(df["lease_left"].clip(0, 99), anchors_years, anchors_rel)

In [28]:
# Loading Aux Data
aux_dict = {
    "hdb": pd.read_csv(hdb_data_path),
    "mrt": pd.read_csv(mrt_stations_path),
    "pri": pd.read_csv(pri_schools_path),
    "sec": pd.read_csv(sec_schools_path),
    "malls": pd.read_csv(shopping_malls_path),
    "hawkers": pd.read_csv(gov_hawkers_path)
}

In [29]:
# HDB Data
hdb = aux_dict["hdb"].copy()

hdb.columns = hdb.columns.str.lower()

for col in ["town","block", "address", "subzone", "planning_area", "region"]:
    if col in hdb.columns: hdb[col] = hdb[col].map(norm_key)

In [30]:
hdb.head()

,town,block,address,postal_code,latitude,longitude,max_floor,subzone,planning_area,region
0,WOODLANDS,205,MARSILING DRIVE,730205,1.444946,103.773484,14,NORTH COAST,WOODLANDS,NORTH REGION
1,CLEMENTI,506,WEST COAST DRIVE,120506,1.312517,103.760867,12,CLEMENTI CENTRAL,CLEMENTI,WEST REGION
2,SENGKANG,267B,COMPASSVALE LINK,542267,1.383200,103.897433,17,COMPASSVALE,SENGKANG,NORTH-EAST REGION
3,WOODLANDS,521,WOODLANDS DRIVE 14,730521,1.433716,103.793326,12,WOODLANDS SOUTH,WOODLANDS,NORTH REGION
4,PASIR RIS,531,PASIR RIS DRIVE 1,510531,1.371362,103.950398,8,PASIR RIS CENTRAL,PASIR RIS,EAST REGION


In [31]:
hdb_pairs = hdb[["town","block"]].drop_duplicates()
train_pairs = train_df[["town","block"]]
test_pairs  = test_df[["town","block"]]

In [32]:
train_cov = train_pairs.merge(hdb_pairs, on=["town","block"], how="left", indicator=True)
test_cov = test_pairs.merge(hdb_pairs,  on=["town","block"], how="left", indicator=True)

train_match_n = (train_cov["_merge"] == "both").sum()
test_match_n = (test_cov["_merge"] == "both").sum()

print(f"[COVERAGE] Train matches: {train_match_n}/{len(train_cov)} = {train_match_n/len(train_cov)}")
print(f"[COVERAGE] TEST matches: {test_match_n}/{len(test_cov)} = {test_match_n/len(test_cov)}")

[COVERAGE] Train matches: 162691/162691 = 1.0
[COVERAGE] TEST matches: 50000/50000 = 1.0


In [33]:
hdb_unique = hdb.drop_duplicates(subset=["town","block"])

cols_to_add = ["latitude","longitude","max_floor","subzone","planning_area","region"]
cols_to_add = [c for c in cols_to_add if c in hdb_unique.columns]  # guard

train_df = train_df.merge(hdb_unique[["town","block", *cols_to_add]], on=["town","block"], how="left")
test_df = test_df.merge (hdb_unique[["town","block", *cols_to_add]], on=["town","block"], how="left")

In [34]:
# Floor effects (relative position in block)
# Higher floors typically command a premium, and “how high” depends on the block’s max floor
# Relative floor and an “is high floor” flag. https://ideas.repec.org/a/taf/apeclt/v26y2019i6p436-439.html

for df in (train_df, test_df):
    df["rel_floor"] = (df["floor_mid"] / df["max_floor"]).replace([np.inf, -np.inf], np.nan)
    df["is_high_floor"] = (df["rel_floor"] >= 0.7).astype(int)

In [35]:
# MRT Data
mrt = aux_dict["mrt"].copy()

mrt.columns = mrt.columns.str.lower()

for col in ["code","name", "status", "subzone", "planning_area", "region"]:
    if col in mrt.columns: mrt[col] = mrt[col].map(norm_key)

In [36]:
mrt

,code,name,status,latitude,longitude,subzone,planning_area,region
0,NS10,ADMIRALTY MRT STATION,OPEN,1.440689,103.800933,WOODLANDS EAST,WOODLANDS,NORTH REGION
1,EW9,ALJUNIED MRT STATION,OPEN,1.316442,103.882981,ALJUNIED,GEYLANG,CENTRAL REGION
2,NS16,ANG MO KIO MRT STATION,OPEN,1.370017,103.849450,CHENG SAN,ANG MO KIO,NORTH-EAST REGION
3,CR11,ANG MO KIO MRT STATION,PLANNED,1.370017,103.849450,CHENG SAN,ANG MO KIO,NORTH-EAST REGION
4,CR2,AVIATION PARK MRT STATION,PLANNED,1.370000,104.003056,CHANGI AIRPORT,CHANGI,EAST REGION
...,...,...,...,...,...,...,...,...
238,NE11,WOODLEIGH MRT STATION,OPEN,1.339167,103.870833,BIDADARI,TOA PAYOH,CENTRAL REGION
239,DT36,XILIN MRT STATION,PLANNED,1.328889,103.965000,XILIN,TAMPINES,EAST REGION
240,NS5,YEW TEE MRT STATION,OPEN,1.396986,103.747239,YEW TEE,CHOA CHU KANG,WEST REGION
241,NS15,YIO CHU KANG MRT STATION,OPEN,1.381906,103.844817,YIO CHU KANG,ANG MO KIO,NORTH-EAST REGION


In [37]:
print(mrt['status'].unique())
# Open and Planned MRT proximity features

from sklearn.neighbors import BallTree

mrt_open = mrt[mrt["status"] == "OPEN"].copy()
mrt_planned = mrt[mrt["status"] == "PLANNED"].copy()

R = 6371000.0
def to_rad(df): return np.deg2rad(df[["latitude", "longitude"]].to_numpy(dtype=float))

tree_open = BallTree(to_rad(mrt_open), metric="haversine")
tree_planned = BallTree(to_rad(mrt_planned), metric="haversine")

['OPEN' 'PLANNED']


In [38]:
def add_mrt_features(df, radii_open=(500,1000,2000), radii_plan=(500,1000,2000)):
    idx = df[["latitude","longitude"]].dropna().index
    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))

    # OPEN: nearest + counts
    if tree_open is not None and len(idx):
        dist, _ = tree_open.query(X, k=1)
        df.loc[idx, "mrt_open_nearest_m"] = dist[:,0] * R
        for r in radii_open:
            df.loc[idx, f"mrt_open_within_{r}m"] = tree_open.query_radius(X, r/R, count_only=True)
    else:
        df["mrt_open_nearest_m"] = np.nan
        for r in radii_open: df[f"mrt_open_within_{r}m"] = 0

    # PLANNED: nearest + counts
    if tree_planned is not None and len(idx):
        dist, _ = tree_planned.query(X, k=1)
        df.loc[idx, "mrt_plan_nearest_m"] = dist[:,0] * R
        for r in radii_plan:
            df.loc[idx, f"mrt_plan_within_{r}m"] = tree_planned.query_radius(X, r/R, count_only=True)
    else:
        df["mrt_plan_nearest_m"] = np.nan
        for r in radii_plan: df[f"mrt_plan_within_{r}m"] = 0

    # Comparative signals
    df["mrt_any_nearest_m"] = np.nanmin(
        np.vstack([df["mrt_open_nearest_m"].to_numpy(dtype=float),
                   df["mrt_plan_nearest_m"].to_numpy(dtype=float)]),
        axis=0
    )
    df["mrt_plan_closer_than_open"] = (
        (df["mrt_plan_nearest_m"] < df["mrt_open_nearest_m"]).astype("Int64")
    )

    # fill NA counts for rows without coords
    for c in [col for col in df.columns if col.startswith(("mrt_open_within_","mrt_plan_within_"))]:
        df[c] = df[c].fillna(0).astype(int)

    return df

train_df = add_mrt_features(train_df)
test_df = add_mrt_features(test_df)

In [39]:
train_df

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,mrt_open_nearest_m,mrt_open_within_500m,mrt_open_within_1000m,mrt_open_within_2000m,mrt_plan_nearest_m,mrt_plan_within_500m,mrt_plan_within_1000m,mrt_plan_within_2000m,mrt_any_nearest_m,mrt_plan_closer_than_open
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,...,306.013744,1,1,4,2269.847356,0,0,0,306.013744,0
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,...,1000.891332,0,0,7,786.297948,0,1,3,786.297948,1
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,...,1052.540106,0,0,3,2144.006007,0,0,0,1052.540106,0
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,...,872.296018,0,1,6,872.296018,0,1,1,872.296018,0
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,...,1342.862156,0,0,3,1912.846578,0,0,1,1342.862156,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162686,2017-07-01,HOUGANG,4 ROOM,708,HOUGANG AVENUE 2,2.0,91.0,NEW GENERATION,1985,335000.0,...,627.321503,0,2,3,627.321503,0,1,3,627.321503,0
162687,2020-09-01,PASIR RIS,4 ROOM,634,PASIR RIS DRIVE 1,5.0,104.0,MODEL A,1995,388000.0,...,1125.824511,0,0,1,759.006005,0,2,4,759.006005,1
162688,2017-10-01,GEYLANG,4 ROOM,319,UBI AVENUE 1,11.0,84.0,SIMPLIFIED,1985,373000.0,...,443.281824,1,3,10,3811.017617,0,0,0,443.281824,0
162689,2020-08-01,SENGKANG,5 ROOM,290B,COMPASSVALE CRESCENT,2.0,110.0,IMPROVED,2001,420000.0,...,637.643152,0,2,6,2214.204658,0,0,0,637.643152,0


In [40]:
# School Data
pri = aux_dict["pri"].copy()
sec = aux_dict["sec"].copy()

pri.columns = pri.columns.str.lower()
sec.columns = sec.columns.str.lower()

for col in ["name","street", "subzone", "planning_area", "region"]:
    if col in pri.columns: pri[col] = pri[col].map(norm_key)
    if col in sec.columns: sec[col] = sec[col].map(norm_key)

In [41]:
pri.head()

,name,street,postal_code,latitude,longitude,subzone,planning_area,region
0,ADMIRALTY PRIMARY SCHOOL,WOODLANDS CIRCLE,738907,1.442635,103.800040,WOODLANDS EAST,WOODLANDS,NORTH REGION
1,AHMAD IBRAHIM PRIMARY SCHOOL,YISHUN STREET 11,768643,1.433153,103.832942,YISHUN WEST,YISHUN,NORTH REGION
2,AI TONG SCHOOL,BRIGHT HILL DRIVE,579646,1.360583,103.833020,UPPER THOMSON,BISHAN,CENTRAL REGION
3,ALEXANDRA PRIMARY SCHOOL,PRINCE CHARLES CRESCENT,159016,1.291334,103.824425,ALEXANDRA NORTH,BUKIT MERAH,CENTRAL REGION
4,ANCHOR GREEN PRIMARY SCHOOL,ANCHORVALE DRIVE,544969,1.390370,103.887165,ANCHORVALE,SENGKANG,NORTH-EAST REGION


In [42]:
sec.head()

,name,street,postal_code,latitude,longitude,subzone,planning_area,region
0,ADMIRALTY SECONDARY SCHOOL,WOODLANDS CRESCENT,737916,1.445891,103.802398,WOODLANDS EAST,WOODLANDS,NORTH REGION
1,AHMAD IBRAHIM SECONDARY SCHOOL,YISHUN AVENUE 7,768928,1.436060,103.829719,YISHUN WEST,YISHUN,NORTH REGION
2,ANDERSON SECONDARY SCHOOL,ANG MO KIO STREET 53,569206,1.374340,103.851554,CHENG SAN,ANG MO KIO,NORTH-EAST REGION
3,ANGLICAN HIGH SCHOOL,UPPER CHANGI ROAD,487012,1.331181,103.941853,BEDOK NORTH,BEDOK,EAST REGION
4,ANGLO-CHINESE SCHOOL (BARKER ROAD),BARKER ROAD,309919,1.319616,103.835773,MALCOLM,NOVENA,CENTRAL REGION


In [43]:
# Distance to school
# MOE’s P1 registration use home-school distance categories of <1 km, 1–2 km, >2 km for priority, so those should capture a policy-driven price signal.
# For secondary school I'm not sure, maybe if they still live there a sec school nearby is a boon?

def add_proximity_to_school(df, tree, prefix, radii=(1000, 2000)):
    idx = df[["latitude","longitude"]].dropna().index
    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))
    # nearest distance (meters)
    dist, _ = tree.query(X, k=1)
    df.loc[idx, f"{prefix}_nearest_m"] = dist[:,0] * R
    # counts within radii (meters)
    for r in radii:
        cnt = tree.query_radius(X, r / R, count_only=True)
        df.loc[idx, f"{prefix}_within_{r}m"] = cnt
    # fill rows without coords
    for r in radii:
        df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
    return df

In [44]:
# Primary schools
tree_pri = BallTree(to_rad(pri), metric="haversine")
train_df = add_proximity_to_school(train_df, tree_pri, "pri", radii=(1000, 2000))
test_df = add_proximity_to_school(test_df, tree_pri, "pri", radii=(1000, 2000))

# Secondary schools
tree_sec = BallTree(to_rad(sec), metric="haversine")
train_df = add_proximity_to_school(train_df, tree_sec, "sec", radii=(1000, 2000))
test_df = add_proximity_to_school(test_df, tree_sec, "sec", radii=(1000, 2000))

In [45]:
train_df.head()

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,mrt_plan_within_1000m,mrt_plan_within_2000m,mrt_any_nearest_m,mrt_plan_closer_than_open,pri_nearest_m,pri_within_1000m,pri_within_2000m,sec_nearest_m,sec_within_1000m,sec_within_2000m
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,...,0,0,306.013744,0,168.451629,4,10,268.534382,2,8
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,...,1,3,786.297948,1,532.113952,1,9,404.943645,3,11
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,...,0,0,1052.540106,0,156.783632,5,7,238.378434,2,5
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,...,1,1,872.296018,0,177.317685,6,15,522.088513,3,7
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,...,0,1,1342.862156,0,812.017772,3,13,594.225657,2,7


In [46]:
mall = aux_dict["malls"].copy()
mall.columns = mall.columns.str.lower()
for col in ["name", "street", "postal_code", "latitude", "longitude", "subzone", "planning_area", "region"]:
    if col in pri.columns: pri[col] = pri[col].map(norm_key)

In [47]:
mall

,name,street,postal_code,latitude,longitude,subzone,planning_area,region
0,Bugis Junction,victoria street,188021,1.299113,103.855411,bugis,downtown core,central region
1,Bugis+,north bridge road,188735,1.298141,103.855635,bugis,downtown core,central region
2,Capitol Piazza,stamford road,178905,1.293079,103.851262,city hall,downtown core,central region
3,Cathay Cineleisure Orchard,grange road,239695,1.301521,103.836430,somerset,orchard,central region
4,The Centrepoint,orchard road,238843,1.301978,103.839759,somerset,orchard,central region
...,...,...,...,...,...,...,...,...
84,Queensway Shopping Centre,queensway,149053,1.287595,103.803397,queensway,queenstown,central region
85,The Rail Mall,upper bukit timah road,678066,1.359569,103.767610,nature reserve,bukit panjang,west region
86,The Star Vista,vista exchange green,138617,1.306970,103.788420,one north,queenstown,central region
87,West Coast Plaza,west coast road,127371,1.303625,103.765858,clementi woods,clementi,west region


In [48]:
def add_proximity_to_mall(df, tree, prefix, radii=(1000, 2000)):
    idx = df[["latitude","longitude"]].dropna().index
    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))
    # nearest distance (meters)
    dist, _ = tree.query(X, k=1)
    df.loc[idx, f"{prefix}_nearest_m"] = dist[:,0] * R
    # counts within radii (meters)
    for r in radii:
        cnt = tree.query_radius(X, r / R, count_only=True)
        df.loc[idx, f"{prefix}_within_{r}m"] = cnt
    # fill rows without coords
    for r in radii:
        df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
    return df

In [49]:
tree_mall = BallTree(to_rad(mall), metric="haversine")
train_df = add_proximity_to_mall(train_df, tree_mall, "mall", radii=(1000, 2000))
test_df = add_proximity_to_mall(test_df, tree_mall, "mall", radii=(1000, 2000))

In [50]:
train_df.head()

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,mrt_plan_closer_than_open,pri_nearest_m,pri_within_1000m,pri_within_2000m,sec_nearest_m,sec_within_1000m,sec_within_2000m,mall_nearest_m,mall_within_1000m,mall_within_2000m
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,...,0,168.451629,4,10,268.534382,2,8,1994.701707,0,1
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,...,1,532.113952,1,9,404.943645,3,11,1176.138670,0,3
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,...,0,156.783632,5,7,238.378434,2,5,980.543397,1,1
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,...,0,177.317685,6,15,522.088513,3,7,463.143097,1,4
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,...,0,812.017772,3,13,594.225657,2,7,851.672489,1,2


In [51]:
hawker = aux_dict["hawkers"].copy()
hawker.columns = hawker.columns.str.lower()

In [52]:
hawker.head()

,name,address,type,owner,number_of_stalls,latitude,longitude
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,1.324160,103.814166
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,1.281278,103.847032
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,1.320350,103.955478
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,1.288831,103.827354
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,1.307344,103.856889


In [53]:
def add_proximity_to_hawker(df, tree, prefix, radii=(500, 1000)):
    idx = df[["latitude","longitude"]].dropna().index
    # nothing to do if no points with coords
    if len(idx) == 0:
        # ensure columns exist for downstream code
        for r in radii:
            df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
            df[f"{prefix}_stalls_within_{r}m"] = df.get(f"{prefix}_stalls_within_{r}m", 0).fillna(0).astype(int)
        df[f"{prefix}_nearest_m"] = np.nan
        return df

    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))
    # nearest distance (meters)
    dist, _ = tree.query(X, k=1)
    df.loc[idx, f"{prefix}_nearest_m"] = dist[:,0] * R

    # counts within radii (meters) and sum of stalls within radii
    stalls_col = 'number_of_stalls'
    for r in radii:
        inds = tree.query_radius(X, r / R)
        cnt = np.array([len(a) for a in inds])
        df.loc[idx, f"{prefix}_within_{r}m"] = cnt
        stalls_sum = np.array([hawker.iloc[a][stalls_col].sum() if len(a) > 0 else 0 for a in inds])
        df.loc[idx, f"{prefix}_stalls_within_{r}m"] = stalls_sum

    # fill rows without coords and ensure integer types for counts
    for r in radii:
        df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
        df[f"{prefix}_stalls_within_{r}m"] = df.get(f"{prefix}_stalls_within_{r}m", 0).fillna(0).astype(int)
    return df

In [54]:
tree_hawker = BallTree(to_rad(hawker), metric="haversine")
train_df = add_proximity_to_hawker(train_df, tree_hawker, "hawker", radii=(500, 1000))
test_df = add_proximity_to_hawker(test_df, tree_hawker, "hawker", radii=(500, 1000))

In [55]:

train_df.groupby("hawker_within_500m").size()

,0
hawker_within_500m,
0,117100
1,29737
2,12248
3,2956
4,582
5,68


In [56]:
train_df

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,sec_within_1000m,sec_within_2000m,mall_nearest_m,mall_within_1000m,mall_within_2000m,hawker_nearest_m,hawker_within_500m,hawker_stalls_within_500m,hawker_within_1000m,hawker_stalls_within_1000m
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,...,2,8,1994.701707,0,1,2683.955240,0,0,0,0
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,...,3,11,1176.138670,0,3,916.936357,0,0,4,340
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,...,2,5,980.543397,1,1,5364.143377,0,0,0,0
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,...,3,7,463.143097,1,4,4563.650903,0,0,0,0
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,...,2,7,851.672489,1,2,2921.355463,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162686,2017-07-01,HOUGANG,4 ROOM,708,HOUGANG AVENUE 2,2.0,91.0,NEW GENERATION,1985,335000.0,...,3,10,828.902514,2,2,911.095177,0,0,1,186
162687,2020-09-01,PASIR RIS,4 ROOM,634,PASIR RIS DRIVE 1,5.0,104.0,MODEL A,1995,388000.0,...,0,1,1670.498928,0,1,3608.220549,0,0,0,0
162688,2017-10-01,GEYLANG,4 ROOM,319,UBI AVENUE 1,11.0,84.0,SIMPLIFIED,1985,373000.0,...,1,3,2534.264925,0,0,886.171063,0,0,1,110
162689,2020-08-01,SENGKANG,5 ROOM,290B,COMPASSVALE CRESCENT,2.0,110.0,IMPROVED,2001,420000.0,...,3,9,594.746802,1,4,4392.346046,0,0,0,0


### lets add age using month

In [57]:
def add_flat_age_days(train_df, test_df):

    train_df['lease_commence_dt'] = pd.to_datetime(train_df['lease_commence_date'].astype(str) + '-01-01')
    test_df['lease_commence_dt'] = pd.to_datetime(test_df['lease_commence_date'].astype(str) + '-01-01')

    train_df['flat_age_days'] = (train_df['month'] - train_df['lease_commence_dt']).dt.days
    test_df['flat_age_days'] = (test_df['month'] - test_df['lease_commence_dt']).dt.days

    train_df = train_df.drop(columns=['lease_commence_dt'])
    test_df = test_df.drop(columns=['lease_commence_dt'])

    return train_df, test_df

train_df, test_df = add_flat_age_days(train_df, test_df)
train_df

,month,town,flat_type,block,street,floor_mid,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,sec_within_2000m,mall_nearest_m,mall_within_1000m,mall_within_2000m,hawker_nearest_m,hawker_within_500m,hawker_stalls_within_500m,hawker_within_1000m,hawker_stalls_within_1000m,flat_age_days
0,2020-10-01,WOODLANDS,4 ROOM,681B,WOODLANDS DRIVE 62,8.0,102.0,PREMIUM APARTMENT,2000,420000.0,...,8,1994.701707,0,1,2683.955240,0,0,0,0,7579
1,2021-07-01,BISHAN,4 ROOM,264,BISHAN STREET 24,8.0,104.0,MODEL A,1992,585000.0,...,11,1176.138670,0,3,916.936357,0,0,4,340,10774
2,2021-05-01,BUKIT PANJANG,4 ROOM,520,JELAPANG ROAD,20.0,102.0,MODEL A,1998,450000.0,...,5,980.543397,1,1,5364.143377,0,0,0,0,8521
3,2021-08-01,PUNGGOL,4 ROOM,121B,EDGEDALE PLAINS,17.0,93.0,MODEL A,2017,465000.0,...,7,463.143097,1,4,4563.650903,0,0,0,0,1673
4,2023-05-01,HOUGANG,5 ROOM,997B,BUANGKOK CRESCENT,11.0,113.0,IMPROVED,2018,710000.0,...,7,851.672489,1,2,2921.355463,0,0,0,0,1946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162686,2017-07-01,HOUGANG,4 ROOM,708,HOUGANG AVENUE 2,2.0,91.0,NEW GENERATION,1985,335000.0,...,10,828.902514,2,2,911.095177,0,0,1,186,11869
162687,2020-09-01,PASIR RIS,4 ROOM,634,PASIR RIS DRIVE 1,5.0,104.0,MODEL A,1995,388000.0,...,1,1670.498928,0,1,3608.220549,0,0,0,0,9375
162688,2017-10-01,GEYLANG,4 ROOM,319,UBI AVENUE 1,11.0,84.0,SIMPLIFIED,1985,373000.0,...,3,2534.264925,0,0,886.171063,0,0,1,110,11961
162689,2020-08-01,SENGKANG,5 ROOM,290B,COMPASSVALE CRESCENT,2.0,110.0,IMPROVED,2001,420000.0,...,9,594.746802,1,4,4392.346046,0,0,0,0,7152


In [58]:
# One-hot for coarse region, and target/mean encoding for higher-cardinality planning_area/subzone

# Region one-hot
region_dum = pd.get_dummies(train_df["region"], prefix="region", dummy_na=False)
train_df = pd.concat([train_df, region_dum], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df["region"], prefix="region", dummy_na=False)], axis=1)
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)  # align after dummies

# KFold mean encoding for planning_area
from sklearn.model_selection import KFold
def kfold_target_mean_encode(tr, te, col, y="resale_price", n_splits=5, seed=777):
    tr = tr.copy(); te = te.copy()
    global_mean = tr[y].mean()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_enc = pd.Series(index=tr.index, dtype=float)
    for tr_idx, val_idx in kf.split(tr):
        m = tr.iloc[tr_idx].groupby(col)[y].mean()
        tr_enc.iloc[val_idx] = tr.iloc[val_idx][col].map(m).fillna(global_mean)
    te_enc = te[col].map(tr.groupby(col)[y].mean()).fillna(global_mean)
    tr[f"{col}_te"] = tr_enc
    te[f"{col}_te"] = te_enc
    return tr, te

train_df, test_df = kfold_target_mean_encode(train_df, test_df, "planning_area")
train_df, test_df = kfold_target_mean_encode(train_df, test_df, "subzone")

In [59]:
# Improving upon flat-type price hierarchy

# interactions
for df in (train_df, test_df):
    df["exec_x_area"] = df["is_exec"] * df["floor_area_sqm"]
    df["multigen_x_area"] = df["is_multigen"] * df["floor_area_sqm"]

# coarse one-hot for model (limit top N categories to avoid sparsity)
top_models = train_df["flat_model"].value_counts().head(8).index
for df in (train_df, test_df):
    df["flat_model_top"] = df["flat_model"].where(df["flat_model"].isin(top_models), "OTHER")
model_dum = pd.get_dummies(train_df["flat_model_top"], prefix="model")
train_df = pd.concat([train_df, model_dum], axis=1)
test_df  = pd.concat([test_df, pd.get_dummies(test_df["flat_model_top"], prefix="model")], axis=1)
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)


### baseline models

linear regression

In [ ]:
target = 'resale_price'

# Define columns that should NOT be used as features
# block and street are in lat lon
# month and lease_commence_date are in flat_age_days
drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']


numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

# Create a validation split (80% train, 20% validation)
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_pipeline.fit(X_train_split, y_train_split)
val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

# retraining on all data
model_pipeline.fit(X, y)

# Save the entire pipeline (preprocessor + model) to a file
model_filename = 'linear_regression.joblib'
joblib.dump(model_pipeline, model_filename)


Validation RMSE: 56649.0188


polynomial degree 2 (stop at 2 because this takes 40+mins already)

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)), # Apply polynomial transform
    ('model', LinearRegression())
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'polynomial_regression.joblib'
joblib.dump(model_pipeline, model_filename)


Validation RMSE: 32251.4299


dt regression

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42)) # Apply Decision Tree model
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'decision_tree.joblib'
joblib.dump(model_pipeline, model_filename)

Validation RMSE: 42299.4341


bagging

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', BaggingRegressor(random_state=42)) # Apply Bagging Regression model
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'bagging.joblib'
joblib.dump(model_pipeline, model_filename)

Validation RMSE: 31659.2727


random forest

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42)) # Apply Random Forest Regression model
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'random_forest.joblib'
joblib.dump(model_pipeline, model_filename)


Validation RMSE: 29868.7537


['random_forest.joblib']

lightgbm

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LGBMRegressor(random_state=42)) # Apply LightGBM Regression model
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'lightgbm.joblib'
joblib.dump(model_pipeline, model_filename)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020437 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3777
[LightGBM] [Info] Number of data points in the train set: 130152, number of used features: 277
[LightGBM] [Info] Start training from score 518727.087977


c:\Users\admin\Desktop\school\MComp_AI\CS5228_Knowledge_Discovery_and_Data_Mining\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Validation RMSE: 34479.3284
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3784
[LightGBM] [Info] Number of data points in the train set: 162691, number of used features: 280
[LightGBM] [Info] Start training from score 518843.001477


['lightgbm.joblib']

XGBoost

In [79]:
def train_xgboost_model(
    train_df,
    target: str = "resale_price",
    drop_cols: Optional[List[str]] = None,
    numerical_features: Optional[List[str]] = None,
    categorical_features: Optional[List[str]] = None,
    val_size: float = 0.2,
    random_state: int = 777,
    use_gpu: bool = True,
    n_trials: int = 30,
    early_stopping_rounds: int = 200,
    max_boost_rounds: int = 5000,
    save_dir: Optional[str] = None,
) -> Dict[str, Any]:


    # Column prep
    drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

    X = train_df.drop(columns=drop_cols, errors="ignore")
    y = train_df[target].to_numpy()

    if numerical_features is None:
        numerical_features = list(X.select_dtypes(include=np.number).columns)
    if categorical_features is None:
        categorical_features = list(X.select_dtypes(include=["object", "category"]).columns)

    if not numerical_features and not categorical_features:
        raise ValueError("No features found. Check drop_cols and dtypes.")

    # Preprocessor
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), numerical_features),
            ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")), ("ohe", ohe)]), categorical_features),
        ],
        remainder="drop",
    )

    X_train, X_val, y_train_raw, y_val_raw = train_test_split(
        X, y, test_size=val_size, random_state=random_state
    )

    X_train_t = preprocessor.fit_transform(X_train)
    X_val_t = preprocessor.transform(X_val)

    # Log-transform target for more Gaussian residuals, this should help with outliers
    y_train = np.log1p(y_train_raw)
    y_val = np.log1p(y_val_raw)

    def make_params(overrides: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "max_depth": 8,
            "min_child_weight": 2.0,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "alpha": 0.0, # L1
            "lambda": 1.0, # L2
            "gamma": 0.0,
            "grow_policy": "lossguide",
        }
        if use_gpu:
            params.update({"device": "cuda", "tree_method": "hist"})
        else:
            params.update({"tree_method": "hist"})
        if overrides:
            lr = overrides.pop("learning_rate", None)
            if lr is not None:
                params["eta"] = lr
            params.update(overrides)
        return params

    # Random sampling
    def sample_params() -> Dict[str, Any]:
        return {
            "learning_rate": 10 ** rng.uniform(-2.3, -0.7), # ~0.005..0.2
            "max_depth": int(rng.randint(4, 13)), # 4..12
            "min_child_weight": float(10 ** rng.uniform(-0.3, 1.0)), # ~0.5..10
            "subsample": float(rng.uniform(0.6, 1.0)),
            "colsample_bytree": float(rng.uniform(0.6, 1.0)),
            "alpha": float(10 ** rng.uniform(-3, 1)), # 0.001..10
            "lambda": float(10 ** rng.uniform(-2, 1)), # 0.01..10
            "gamma": float(10 ** rng.uniform(-3, 0)), # 0.001..1
            "max_leaves": int(rng.randint(16, 512)),
        }

    # Helpers to use "best trees"
    def best_ntrees(booster: xgb.Booster) -> int:
        if hasattr(booster, "best_iteration") and booster.best_iteration is not None:
            return int(booster.best_iteration) + 1
        if hasattr(booster, "best_ntree_limit") and booster.best_ntree_limit:
            return int(booster.best_ntree_limit)
        return max_boost_rounds

    def predict_best(booster: xgb.Booster, dmat: xgb.DMatrix) -> np.ndarray:
        nbest = best_ntrees(booster)
        try:
            return booster.predict(dmat, iteration_range=(0, nbest))
        except TypeError:
            return booster.predict(dmat, ntree_limit=nbest)

    dtrain = xgb.DMatrix(X_train_t, label=y_train)
    dval = xgb.DMatrix(X_val_t, label=y_val)

    # Hyperparameter search
    best: Dict[str, Any] = {"rmse": math.inf, "params": None, "best_iteration": None, "booster": None}

    pbar = tqdm(range(n_trials), unit="trial", desc="XGB hyperparam search")
    for _ in pbar:
        trial = sample_params()
        params = make_params(trial)
        booster = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=max_boost_rounds,
            evals=[(dval, "val")],
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False,
        )
        # score on original scale
        val_pred = np.expm1(predict_best(booster, dval))
        rmse = float(np.sqrt(mean_squared_error(y_val_raw, val_pred)))

        if rmse < best["rmse"]:
            best.update({
                "rmse": rmse,
                "params": params,
                "best_iteration": best_ntrees(booster) - 1,  # store 0-based
                "booster": booster,
            })
        pbar.set_postfix(rmse=f"{rmse:.2f}", best=f"{best['rmse']:.2f}")

    # Refit with best params
    X_full_t = preprocessor.transform(X)
    y_full = np.log1p(y)
    dfull = xgb.DMatrix(X_full_t, label=y_full)

    final_params = dict(best["params"])
    final_rounds = int(best["best_iteration"]) + 1

    bst_final = xgb.train(
        params=final_params,
        dtrain=dfull,
        num_boost_round=final_rounds,
        verbose_eval=False,
    )

    # Adapter for the prediction
    class BoosterAdapter:
        def __init__(self, booster: xgb.Booster, ntrees: int):
            self.booster = booster
            self.best_iteration = ntrees - 1
        def predict(self, Xmat) -> np.ndarray:
            dmat = xgb.DMatrix(Xmat)
            try:
                return self.booster.predict(dmat, iteration_range=(0, self.best_iteration + 1))
            except TypeError:
                return self.booster.predict(dmat, ntree_limit=self.best_iteration + 1)
        def __repr__(self) -> str:
            return f"BoosterAdapter(best_iteration={self.best_iteration})"

    model_adapter = BoosterAdapter(bst_final, final_rounds)

    if save_dir:
        path = Path(save_dir)
        path.mkdir(parents=True, exist_ok=True)
        joblib.dump(preprocessor, path / "xgb_model.joblib")
        bst_final.save_model(str(path / "xgb_model.json"))
        with (path / "best_summary.json").open("w") as f:
            json.dump(
                {
                    "val_rmse": best["rmse"],
                    "best_iteration": int(best["best_iteration"]),
                    "num_boost_round": final_rounds,
                    "best_params": {
                        k: final_params[k]
                        for k in [
                            "eta", "max_depth", "min_child_weight", "subsample",
                            "colsample_bytree", "alpha", "lambda", "gamma",
                            "grow_policy", *(["device"] if "device" in final_params else []),
                            "tree_method",
                        ]
                        if k in final_params
                    },
                },
                f,
                indent=2,
            )

    return {
        "preprocessor": preprocessor,
        "model": model_adapter,
        "best_params": final_params,
        "best_iteration": int(best["best_iteration"]),
        "val_rmse": float(best["rmse"]),
    }

In [80]:
result = train_xgboost_model(
    train_df,
    target="resale_price",
    drop_cols=["resale_price", "month", "block", "street", "lease_commence_date"],
    save_dir="models/xgb_best",
    n_trials=40,
    use_gpu=True,
)
print(f"Best val RMSE: {result['val_rmse']:.2f}")

XGB hyperparam search:   0%|          | 0/40 [00:00<?, ?trial/s]

Best val RMSE: 25566.88


### prediction

In [81]:
loaded_model = joblib.load("xgb_model.joblib")
drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']
X_sub = test_df.drop(columns=["resale_price", "month", "block", "street", "lease_commence_date"], errors="ignore")
X_sub_t = result["preprocessor"].transform(X_sub)
final_predictions = np.expm1(result["model"].predict(X_sub_t))
results_df = pd.DataFrame({'id': test_df.index,
                           'Predicted': final_predictions})
results_df.to_csv("submission.csv")

In [ ]:
loaded_model = joblib.load("random_forest.joblib") # load saved model

X_submission_test = test_df[X.columns]

final_predictions = loaded_model.predict(X_submission_test)

results_df = pd.DataFrame({
    'id': test_df.index,
    'predicted_resale_price': final_predictions
})

results_df

,id,predicted_resale_price
0,0,521208.96
1,1,611036.64
2,2,402893.04
3,3,440106.64
4,4,514306.64
...,...,...
49995,49995,343973.00
49996,49996,427465.52
49997,49997,519874.25
49998,49998,582948.88
